In [2]:
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os.path

from os import getcwd
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import savemat, loadmat
import math
from scipy.fftpack import fft, ifft
from scipy.stats import zscore, f_oneway
#import samplerate
import multiprocessing as mp
from scipy.signal import resample
import random
import pyreadr
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score
import scipy
import matplotlib.patches as mpatches
import nibabel as nib
import plotly
import plotly.graph_objs as go
import random

In [2]:
stn_file = hdf5storage.loadmat("atlas_index.mat")
stn_vertices = stn_file['atlases'][0]['roi'][0][1][0]['fv'][0]['vertices']
stn_faces = stn_file['atlases'][0]['roi'][0][1][0]['fv'][0]['faces']
stn_faces = stn_faces.astype(int) - 1

stn_vertices_motor = stn_file['atlases'][0]['roi'][1][1][0]['fv'][0]['vertices']
stn_faces_motor = stn_file['atlases'][0]['roi'][1][1][0]['fv'][0]['faces']
stn_faces_motor = stn_faces_motor.astype(int) - 1

stn_vertices_associative = stn_file['atlases'][0]['roi'][2][1][0]['fv'][0]['vertices']
stn_faces_associative = stn_file['atlases'][0]['roi'][2][1][0]['fv'][0]['faces']
stn_faces_associative = stn_faces_associative.astype(int) - 1

stn_vertices_limbic = stn_file['atlases'][0]['roi'][3][1][0]['fv'][0]['vertices']
stn_faces_limbic = stn_file['atlases'][0]['roi'][3][1][0]['fv'][0]['faces']
stn_faces_limbic = stn_faces_limbic.astype(int) - 1

In [4]:
i_stn, j_stn, k_stn = stn_faces.T
x_stn, y_stn, z_stn = stn_vertices.T

i_stn_motor, j_stn_motor, k_stn_motor = stn_faces_motor.T
x_stn_motor, y_stn_motor, z_stn_motor = stn_vertices_motor.T

i_stn_associative, j_stn_associative, k_stn_associative = stn_faces_associative.T
x_stn_associative, y_stn_associative, z_stn_associative = stn_vertices_associative.T

i_stn_limbic, j_stn_limbic, k_stn_limbic = stn_faces_limbic.T
x_stn_limbic, y_stn_limbic, z_stn_limbic = stn_vertices_limbic.T

In [3]:
mer_loc = pd.read_csv('mer_loc.csv', index_col=0)

cue_unit = pd.read_csv('cue_unit.csv', index_col=0)
cue_unit.shape

(229, 15)

In [17]:
save_all_unit = mer_loc.iloc[22:, -6:-3]
save_all_unit['new'] = 1
save_all_unit['new2'] = 1
save_all_unit['loc'] = 'STN'
save_all_unit = np.array(save_all_unit)
np.savetxt('template/all_unit_loc.txt', save_all_unit, delimiter=" ", newline = "\n", fmt="%s")


In [16]:
unit_cue_pos = cue_unit[cue_unit['Cue dir'] == "+"].index
unit_cue_neg = cue_unit[cue_unit['Cue dir'] == "-"].index

unit_cue_pos = mer_loc.iloc[unit_cue_pos].iloc[:, 10:13]
unit_cue_neg = mer_loc.iloc[unit_cue_neg].iloc[:, 10:13]
unit_cue = pd.concat([unit_cue_pos, unit_cue_neg], axis=0)

unit_cue.shape

(86, 3)

In [21]:
kde = stats.gaussian_kde(unit_cue.T)
density = kde(unit_cue.T)

In [24]:
# Create a regular 3D grid with 50 points in each dimension
xmin, ymin, zmin = unit_cue.min(axis=0)
xmax, ymax, zmax = unit_cue.max(axis=0)
xi, yi, zi = np.mgrid[xmin:xmax:50j, ymin:ymax:50j, zmin:zmax:50j]

# Evaluate the KDE on a regular grid...
coords = np.vstack([item.ravel() for item in [xi, yi, zi]])
density = kde(coords).reshape(xi.shape)

In [27]:
coords.shape
density.shape

(3, 125000)

(50, 50, 50)

In [30]:
stn_file = h5py.File('atlas_index.mat', 'r')
stn_full_affine = np.array(stn_file[stn_file['atlases']['roi'][1, 0]]['nii']['mat']).T


In [31]:
density = nib.Nifti1Image(density, stn_full_affine)
nib.save(density, 'template/density.nii.gz')

In [19]:
# Cue specific location

cue_only = []
cue_speech = []
for item in range(cue_unit.shape[0]):
    if cue_unit.iloc[item, 2] == "After":
        if cue_unit.iloc[item, 5] == "Before":
            cue_speech.append(item)
        elif cue_unit.iloc[item, 5] == "After":
            cue_speech.append(item)
        else:
            cue_only.append(item)
            
cue_only = mer_loc.iloc[cue_only].iloc[:, 10:13]
cue_speech = mer_loc.iloc[cue_speech].iloc[:, 10:13]

cue_only_kde = stats.gaussian_kde(cue_only.T)
cue_only_density = cue_only_kde(cue_only.T)

# Create a regular 3D grid with 50 points in each dimension
xmin, ymin, zmin = cue_only.min(axis=0)
xmax, ymax, zmax = cue_only.max(axis=0)
xi, yi, zi = np.mgrid[xmin:xmax:50j, ymin:ymax:50j, zmin:zmax:50j]

# Evaluate the KDE on a regular grid...
coords = np.vstack([item.ravel() for item in [xi, yi, zi]])
cue_only_density = cue_only_kde(coords).reshape(xi.shape)

In [37]:
cue_only_density = nib.Nifti1Image(cue_only_density, stn_full_affine)
nib.save(cue_only_density, 'template/cue_only_density.nii.gz')

In [21]:
cue_only['new'] = 1
cue_only['new2'] = 1
cue_only['loc'] = 'STN'
cue_only = np.array(cue_only)
np.savetxt('template/cue_only.txt', cue_only, delimiter=" ", newline = "\n", fmt="%s")


In [22]:
# STN location that elicited STG EP

unique_stn_mni = [[-14.98387934, -12.92955243,  -5.49369781],
       [-14.70173477, -12.72993921,  -6.9148277 ],
       [-14.61559584, -13.08360543,  -6.69066031],
       [-14.30137752, -15.52726452,  -6.05530294],
       [-14.03616376, -13.9636122 ,  -8.70867027],
       [-14.00005879, -14.74497848,  -8.96395878],
       [-13.73481012, -13.47723165,  -5.39605118],
       [-13.56196624, -15.93644309,  -7.66050696],
       [-13.37473971, -17.41903282,  -6.20454728],
       [-13.31060832, -12.20960832,  -5.93658854],
       [-13.29565498, -15.71861443,  -6.72678909],
       [-13.23565603, -15.46435028,  -8.16041817],
       [-13.18059202, -14.17237787,  -8.08707092],
       [-12.17040858, -14.08359043,  -8.36022743],
       [-12.03716398, -14.73622513,  -7.1264492 ],
       [-10.9304167 , -14.53749422,  -5.21299281]]

unique_stn_kde = stats.gaussian_kde(np.array(unique_stn_mni).T)
unique_stn_density = cue_only_kde(np.array(unique_stn_mni).T)

# Create a regular 3D grid with 50 points in each dimension
xmin, ymin, zmin = np.array(unique_stn_mni).min(axis=0)
xmax, ymax, zmax = np.array(unique_stn_mni).max(axis=0)
xi, yi, zi = np.mgrid[xmin:xmax:50j, ymin:ymax:50j, zmin:zmax:50j]

# Evaluate the KDE on a regular grid...
coords = np.vstack([item.ravel() for item in [xi, yi, zi]])
unique_stn_density = unique_stn_kde(coords).reshape(xi.shape)

In [45]:
unique_stn_density = nib.Nifti1Image(unique_stn_density, stn_full_affine)
nib.save(unique_stn_density, 'template/unique_stn_density.nii.gz')

In [24]:
unique_stn_mni = pd.DataFrame(unique_stn_mni)
unique_stn_mni['new'] = 1
unique_stn_mni['new2'] = 1
unique_stn_mni['loc'] = 'STN'
unique_stn_mni = np.array(unique_stn_mni)
np.savetxt('template/unique_stn_mni.txt', unique_stn_mni, delimiter=" ", newline = "\n", fmt="%s")
